In [128]:
'''
Created on 2016. 11. 18

@author: dato
@desc: cnlp hw7 (naive bayes)
'''
#-*- coding: utf-8 -*-

import io
from konlpy.tag import Kkma
from konlpy.utils import pprint
import math

In [16]:
data_file = './wsd_dataset.tsv'

In [17]:
f = io.open(data_file, 'r', encoding='utf-8')
lines = f.readlines()

total_doc = len(lines)
parsed_lines = [line.split('\t') for line in lines]

print 'total # of doc : ' + str(total_doc)

total # of doc : 4937


In [18]:
pprint (lines[3000])

배__09/NNG	그것/NP 도/JX 이른바/MAG 선진국/NNG 이/VCP 라/EC 하/VX 는/ETM 경제/NNG 협력/NNG 개발/NNG 기구/NNG (/SS OECD/SL )/SS 의/JKG 평균/NNG 을/JKO 10/SN 	배/NNG	나/JC 능가/NNG 하/XSV 는/ETM 수준/NNG 이/VCP 라고/EC 하/VV ᆫ다/EF ./SF 



# split data set

사과 05 (apple), 08 (apology) 

배 01 (stomach), 02 (ship), 03 (pear), 09 (double)

감자 01 (potato), 05 (capital reduction)

In [432]:
apple = []
pear = []
potato = []

for parsed_line in parsed_lines:
    tmp = parsed_line[0].split('_')[0]    
    
    if( tmp == unicode('사과'.decode('utf-8'))):
        apple.append(parsed_line)
    elif( tmp == unicode('배'.decode('utf-8'))):
        pear.append(parsed_line)
    else:
        potato.append(parsed_line)

In [433]:
print len(apple)
print len(pear)
print len(potato)

pprint (apple[1])

742
3868
327
[사과__08/NNG,
 그/MM 후/NNG 자정/NNG 을/JKO 넘기/VV ᆫ/ETM 시각/NNG 에/JKB 일본/NNP 측/NNB 은/JX "/SS ,
 사과/NNG,
 성명/NNG 은/JX 여러/MM 사정/NNG 으로/JKB 발표/NNG 하/XSV 기/ETN 어렵/VA 다/EC "/SS 는/ETM 내용/NNG 을/JKO 일방/NNG 통보/NNG 하/XSV ᆫ/ETM 채/NNB 모든/MM 것/NNB ᆯ/JKO 마무리/NNG 하/VV 아/EC 
]


In [434]:
apple_train, apple_test = random_sample(apple, 0.2)
pear_train, pear_test = random_sample(pear, 0.2)
potato_train, potato_test = random_sample(potato, 0.2)

# util func

In [435]:
# random sampling data from list

from random import randrange
import random

def random_sample(list_data, portion):
    
    list_train = []
    list_test = []
    a = random.sample(xrange(len(list_data)), int(len(list_data)*portion))

    for x in range(len(list_data)):
        if x not in a:
            list_train.append(list_data[x])
        else:
            list_test.append(list_data[x])
    
    return list_train, list_test

In [436]:
def add_token_to_dic(dic, tokens):
    for token in tokens:
        if dic.has_key(token):
            dic[token] = dic[token] + 1
        else:
            dic[token] = 1    

In [437]:
def gen_base_dic(dic, tokens):
    for token in tokens:
        if not dic.has_key(token):
            dic[token] = 1

In [438]:
def add_dic(dic_n, base_dic):
   
    for key in base_dic.keys():
        if dic_n.has_key(key):
            dic_n[key] =  dic_n[key] + 1
        else:
            dic_n[key] =  1

In [439]:
def norm_log_dic(dic_n):

    new_dic = {}
    
    N = float(sum(dic_n.values()))
    
    for key in dic_n.keys():
        new_dic[key] =  math.log(dic_n[key] / N)
        
    return new_dic

In [440]:
def cal_score(dic_n, tokens):
    
    score = 0
    
    for token in tokens:
        if dic_n.has_key(token):
            score = score + dic_n[token]
    
    return score

# apple case - training

사과 05 (apple), 08 (apology) 

In [441]:
print len(apple_train)
print len(apple_test)


dic_A = {}
dic_B = {}
base_dic = {}

prior_A = 0
prior_B = 0

for data in apple_train:
    
    gen_base_dic(base_dic, data[1].split(' '))
    gen_base_dic(base_dic, data[3].split(' '))
        
    # class A case
    if data[0].split('__')[1].split('/')[0] == unicode('05'.decode('utf-8')) :
        add_token_to_dic(dic_A, data[1].split(' '))
        add_token_to_dic(dic_A, data[3].split(' '))       
        prior_A = prior_A + 1
        
    # class B case
    else:
        add_token_to_dic(dic_B, data[1].split(' '))
        add_token_to_dic(dic_B, data[3].split(' '))     
        prior_B = prior_B + 1
        
        
# laplace smoothing
add_dic(dic_A, base_dic)
add_dic(dic_B, base_dic)

# normalizing & apply log
norm_log_dic_A = norm_log_dic(dic_A)
norm_log_dic_B = norm_log_dic(dic_B)

# cal log prior
log_prior_A = math.log( prior_A / (float)(prior_A + prior_B) )
log_prior_B = math.log( prior_B / (float)(prior_A + prior_B) )

594
148


# apple case - evaluation

In [442]:
score = 0

print str('Class A \tClass B \tPredict\tLabel')

for data in apple_test:
    
    ret = ''
    predict = ''
    label = ''
    
    # class A case
    nll_A = cal_score(norm_log_dic_A, data[1].split(' '))
    nll_A = nll_A + cal_score(norm_log_dic_A, data[3].split(' '))
    
    nll_B = cal_score(norm_log_dic_B, data[1].split(' '))
    nll_B = nll_B + cal_score(norm_log_dic_B, data[3].split(' '))
    
    final_A = -1 * (nll_A + log_prior_A)
    final_B = -1 * (nll_B + log_prior_B)
    
    ret += (str(final_A) + '\t' + str(final_B))
    ret += '\t'    
    
    # class A case
    if( final_A < final_B ):
        predict = '05'
    else:
        predict = '08'
        
    label = data[0].split('__')[1].split('/')[0]   
    
    if label == predict:
        score = score + 1

    ret += (predict + '\t' + label)
        
    print ret   


Class A 	Class B 	Predict	Label
232.244821142	211.019442415	08	08
178.336623404	169.778179504	08	08
239.798018394	258.934222794	05	05
179.024414036	180.768284607	05	05
188.158766857	176.029148635	08	08
212.428398676	204.657820878	08	08
168.832044565	160.987368174	08	08
259.045323577	239.714793771	08	08
149.77953236	148.123846921	08	08
199.251110623	178.114751727	08	08
337.301600007	300.98524331	08	08
165.030083281	158.573348951	08	08
264.203693406	241.221516367	08	08
181.808867873	175.880174057	08	08
133.469053416	126.041571617	08	08
274.84046776	241.469183001	08	08
221.719191551	203.268364441	08	08
247.126961263	227.964588327	08	08
184.382932037	172.7268875	08	08
108.193722865	100.666982244	08	08
185.458386676	175.977851781	08	08
244.041371894	231.605027323	08	08
166.493991464	150.468004379	08	08
363.914988404	309.927624867	08	08
280.071003984	248.309274013	08	08
148.384672029	146.024095551	08	08
184.408886957	171.60292209	08	08
239.966467151	220.35098303	08	08
213.109713323	189.60314

In [396]:
print 'accuracy=' + str(score / float(len(apple_test)))

accuracy=0.939189189189


# pear case - training

배 01 (stomach), 02 (ship), 03 (pear), 09 (double)

In [448]:
print len(pear_train)
print len(pear_test)

dic_A = {}
dic_B = {}
dic_C = {}
dic_D = {}
base_dic = {}

prior_A = 0
prior_B = 0
prior_C = 0
prior_D = 0

for data in pear_train:
    
    gen_base_dic(base_dic, data[1].split(' '))
    gen_base_dic(base_dic, data[3].split(' '))
        
    # class A case
    if data[0].split('__')[1].split('/')[0] == unicode('01'.decode('utf-8')) :
        add_token_to_dic(dic_A, data[1].split(' '))
        add_token_to_dic(dic_A, data[3].split(' '))       
        prior_A = prior_A + 1
        
    # class B case
    elif data[0].split('__')[1].split('/')[0] == unicode('02'.decode('utf-8')) :
        add_token_to_dic(dic_B, data[1].split(' '))
        add_token_to_dic(dic_B, data[3].split(' '))     
        prior_B = prior_B + 1
        
    # class C case
    elif data[0].split('__')[1].split('/')[0] == unicode('03'.decode('utf-8')) :
        add_token_to_dic(dic_C, data[1].split(' '))
        add_token_to_dic(dic_C, data[3].split(' '))     
        prior_C = prior_C + 1
        
    # class D case
    else :
        add_token_to_dic(dic_D, data[1].split(' '))
        add_token_to_dic(dic_D, data[3].split(' '))     
        prior_D = prior_D + 1
        
        
# laplace smoothing
add_dic(dic_A, base_dic)
add_dic(dic_B, base_dic)
add_dic(dic_C, base_dic)
add_dic(dic_D, base_dic)

# normalizing & apply log
norm_log_dic_A = norm_log_dic(dic_A)
norm_log_dic_B = norm_log_dic(dic_B)
norm_log_dic_C = norm_log_dic(dic_C)
norm_log_dic_D = norm_log_dic(dic_D)

# cal log prior
log_prior_A = math.log( prior_A / (float)(prior_A + prior_B + prior_C + prior_D) )
log_prior_B = math.log( prior_B / (float)(prior_A + prior_B + prior_C + prior_D) )
log_prior_C = math.log( prior_C / (float)(prior_A + prior_B + prior_C + prior_D) )
log_prior_D = math.log( prior_D / (float)(prior_A + prior_B + prior_C + prior_D) )

3095
773


# pear case - evaluation

In [449]:
score = 0

print str('Class A \tClass B \tClass C \tClass D \tPredict\tLabel')

for data in pear_test:
    
    ret = ''
    predict = ''
    label = ''
    
    # class A case
    nll_A = cal_score(norm_log_dic_A, data[1].split(' '))
    nll_A = nll_A + cal_score(norm_log_dic_A, data[3].split(' '))
    
    # class B case
    nll_B = cal_score(norm_log_dic_B, data[1].split(' '))
    nll_B = nll_B + cal_score(norm_log_dic_B, data[3].split(' '))
    
    # class C case
    nll_C = cal_score(norm_log_dic_C, data[1].split(' '))
    nll_C = nll_C + cal_score(norm_log_dic_C, data[3].split(' '))
    
    # class D case
    nll_D = cal_score(norm_log_dic_D, data[1].split(' '))
    nll_D = nll_D + cal_score(norm_log_dic_D, data[3].split(' '))
    
    final_A = -1 * (nll_A + log_prior_A)
    final_B = -1 * (nll_B + log_prior_B)
    final_C = -1 * (nll_C + log_prior_C)
    final_D = -1 * (nll_D + log_prior_D)
    
    ret += (str(final_A) + '\t' + str(final_B) + '\t' + str(final_C) + '\t' + str(final_D))
    ret += '\t'    
    
    if min([final_A, final_B, final_C, final_D]) == final_A:
        predict = '01'
    elif min([final_A, final_B, final_C, final_D]) == final_B:
        predict = '02'
    elif min([final_A, final_B, final_C, final_D]) == final_C:
        predict = '03'
    else:
        predict = '09'
        
    label = data[0].split('__')[1].split('/')[0]   
    
    if label == predict:
        score = score + 1

    ret += (predict + '\t' + label)
        
    print ret   


Class A 	Class B 	Class C 	Class D 	Predict	Label
115.842505347	112.575311442	118.727169832	93.0857338585	09	09
178.433886007	181.251910489	193.853518766	176.552562876	09	09
153.810864982	156.702715938	170.600562237	159.673555854	01	01
319.992917447	324.353467502	340.429083229	305.889182995	09	09
235.526789517	228.340285523	232.29535932	210.860296174	09	09
393.938157157	384.120424449	388.753848139	326.502298545	09	09
363.147124552	353.270559144	370.657264008	296.189268875	09	09
340.29035962	336.448198536	345.720551122	316.163646047	09	09
193.810950016	195.148426855	209.00402802	176.550386381	09	09
216.303103615	215.083830954	230.580737376	197.540981247	09	09
332.550659404	329.763515753	353.246000305	300.138964683	09	09
181.661152277	188.463114892	206.356810046	192.078491762	01	01
377.690304265	362.811299529	381.754674025	333.924037227	09	09
244.287571917	243.738065391	250.302232039	216.971711762	09	09
188.232898591	182.740135926	199.599370617	169.264400232	09	09
183.187685356	188.69696

In [450]:
print 'accuracy=' + str(score / float(len(pear_test)))

accuracy=0.80724450194


# potato case - training

감자 01 (potato), 05 (capital reduction)

In [446]:
print len(potato_train)
print len(potato_test)


dic_A = {}
dic_B = {}
base_dic = {}

prior_A = 0
prior_B = 0

for data in potato_train:
    
    gen_base_dic(base_dic, data[1].split(' '))
    gen_base_dic(base_dic, data[3].split(' '))
        
    # class A case
    if data[0].split('__')[1].split('/')[0] == unicode('01'.decode('utf-8')) :
        add_token_to_dic(dic_A, data[1].split(' '))
        add_token_to_dic(dic_A, data[3].split(' '))       
        prior_A = prior_A + 1
        
    # class B case
    else:
        add_token_to_dic(dic_B, data[1].split(' '))
        add_token_to_dic(dic_B, data[3].split(' '))     
        prior_B = prior_B + 1
        
        
# laplace smoothing
add_dic(dic_A, base_dic)
add_dic(dic_B, base_dic)

# normalizing & apply log
norm_log_dic_A = norm_log_dic(dic_A)
norm_log_dic_B = norm_log_dic(dic_B)

# cal log prior
log_prior_A = math.log( prior_A / (float)(prior_A + prior_B) )
log_prior_B = math.log( prior_B / (float)(prior_A + prior_B) )

262
65


# potato evalation

In [447]:
score = 0

print str('Class A \tClass B \tPredict\tLabel')

for data in potato_test:
    
    ret = ''
    predict = ''
    label = ''
    
    # class A case
    nll_A = cal_score(norm_log_dic_A, data[1].split(' '))
    nll_A = nll_A + cal_score(norm_log_dic_A, data[3].split(' '))
    
    nll_B = cal_score(norm_log_dic_B, data[1].split(' '))
    nll_B = nll_B + cal_score(norm_log_dic_B, data[3].split(' '))
    
    final_A = -1 * (nll_A + log_prior_A)
    final_B = -1 * (nll_B + log_prior_B)
    
    ret += (str(final_A) + '\t' + str(final_B))
    ret += '\t'    
    
    # class A case
    if( final_A < final_B ):
        predict = '01'
    else:
        predict = '05'
        
    label = data[0].split('__')[1].split('/')[0]   
    
    if label == predict:
        score = score + 1

    ret += (predict + '\t' + label)
        
    print ret   


Class A 	Class B 	Predict	Label
243.367805408	250.32385859	01	05
320.137076137	310.089824704	05	05
348.732827228	318.56169045	05	05
320.82785418	329.143022658	01	01
94.228273547	109.672971552	01	01
108.799415123	136.633785108	01	01
75.9027626141	91.9328225338	01	01
267.41984002	300.950780041	01	01
113.48854528	135.720833277	01	01
152.241789724	185.09096749	01	01
184.659897993	217.082910463	01	01
64.9071502275	78.6649302779	01	01
133.376889035	155.682906712	01	01
216.111019218	213.439028009	05	01
216.062176001	228.888203551	01	05
147.69688446	164.298144085	01	01
108.794318316	126.157409765	01	01
70.9312513331	85.4202212611	01	01
70.9312513331	85.4202212611	01	01
162.474365388	194.71108212	01	01
150.053265816	179.873057898	01	01
167.043017078	204.776858988	01	01
122.026325904	142.869462663	01	01
107.226383169	128.302046376	01	01
171.460137678	203.386844679	01	01
73.1229579851	89.5409990804	01	01
162.992639895	189.842891121	01	01
127.754793164	155.706739802	01	01
122.618755205	152.0674614

In [431]:
print 'accuracy=' + str(score / float(len(potato_test)))

accuracy=0.923076923077
